In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from My_Loss import HardTripletLoss2
from My_Loss import HardTripletLoss_D
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 100000
TEST_EPISODE = 1000
LEARNING_RATE =1e-3
Weight_Deacy = 1e-6
GPU = 0
Margin = 1.5

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'CUB1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(312,1200,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1200,2048)
#triplet_network = TripletNetwork(attribute_network, metric_network)  #metric learning   
attribute_network.cuda(GPU) 
metric_network.cuda(GPU)
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.SGD(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy,momentum=0.5)
metric_network_optim = torch.optim.SGD(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=10000 , gamma=1)
metric_network_scheduler = StepLR(metric_network_optim , step_size=10000 , gamma=1)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024

        #print(batch_features_ext)

        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes,cos_sim = False):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = (Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = (attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)

        if cos_sim:
            relations = F.cosine_similarity(batch_features_ext, sample_features_ext).view(-1, class_num)
        else:
            relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    

    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy

In [8]:
print("training...")
last_accuracy = 0.0
loss_zero_num = 0
#writer = SummaryWriter()
for episode in range(EPISODE):
    #attribute_network.train()
    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)
    

    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss_D(margin = Margin).cuda(GPU)
    triplet_loss= criterion((attribute_network(batch_attributes_ext)), 
                            (batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    metric_network.zero_grad()
    attribute_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()
    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        #writer.add_scalar('data/loss_zero_number', loss_zero_num, episode)
        loss_zero_num = 0
        #writer.add_scalar('data/loss', triplet_loss, episode)
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy_per_class(test_features, test_label, np.arange(200), attributes)
        gzsl_seen_accuracy  = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(200), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
#writer.close()

training...
episode: 1 loss tensor(22.4386, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.02
gzsl: unseen=0.0200 , seen=0.0000 , h=0.0000
____________________________________________________________________________________________________
episode: 200 loss tensor(3.6570, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.05867231638418078
gzsl: unseen=0.0530 , seen=0.0106 , h=0.0177
____________________________________________________________________________________________________
episode: 400 loss tensor(2.9636, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.06949378531073445
gzsl: unseen=0.0311 , seen=0.0414 , h=0.0355
____________________________________________________________________________________________________
episode: 600 loss tensor(2.4692, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
cl

episode: 5600 loss tensor(1.2237, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.11745193476175247
gzsl: unseen=0.1175 , seen=0.0443 , h=0.0643
____________________________________________________________________________________________________
episode: 5800 loss tensor(1.5216, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.14503724549621577
gzsl: unseen=0.1437 , seen=0.0432 , h=0.0664
____________________________________________________________________________________________________
episode: 6000 loss tensor(1.4905, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.13641694915254238
gzsl: unseen=0.1354 , seen=0.0553 , h=0.0786
____________________________________________________________________________________________________
episode: 6200 loss tensor(1.5360, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num

episode: 11200 loss tensor(1.1763, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.11189265536723164
gzsl: unseen=0.1116 , seen=0.0148 , h=0.0261
____________________________________________________________________________________________________
episode: 11400 loss tensor(1.4577, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.12956497175141243
gzsl: unseen=0.1296 , seen=0.0145 , h=0.0261
____________________________________________________________________________________________________
episode: 11600 loss tensor(1.2338, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.13522033898305086
gzsl: unseen=0.1352 , seen=0.0214 , h=0.0369
____________________________________________________________________________________________________
episode: 11800 loss tensor(1.2913, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class

zsl: 0.14780225988700565
gzsl: unseen=0.1478 , seen=0.0103 , h=0.0192
____________________________________________________________________________________________________
episode: 16800 loss tensor(1.6966, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.09548022598870055
gzsl: unseen=0.0955 , seen=0.0049 , h=0.0094
____________________________________________________________________________________________________
episode: 17000 loss tensor(1.3576, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.09252542372881356
gzsl: unseen=0.0925 , seen=0.0087 , h=0.0158
____________________________________________________________________________________________________
episode: 17200 loss tensor(1.4986, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.13082485875706215
gzsl: unseen=0.1308 , seen=0.0049 , h=0.0095
_________________________________

episode: 22200 loss tensor(1.2885, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.15065536723163842
gzsl: unseen=0.1507 , seen=0.0095 , h=0.0178
____________________________________________________________________________________________________
episode: 22400 loss tensor(1.3740, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.12228248587570624
gzsl: unseen=0.1223 , seen=0.0164 , h=0.0290
____________________________________________________________________________________________________
episode: 22600 loss tensor(1.3502, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.08958757062146892
gzsl: unseen=0.0896 , seen=0.0060 , h=0.0112
____________________________________________________________________________________________________
episode: 22800 loss tensor(1.3701, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class

...

zsl: 0.17504562633937268
gzsl: unseen=0.1733 , seen=0.0167 , h=0.0305
____________________________________________________________________________________________________
episode: 77400 loss tensor(1.2586, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.16823300214299633
gzsl: unseen=0.1679 , seen=0.0078 , h=0.0149
____________________________________________________________________________________________________
episode: 77600 loss tensor(1.0969, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.12138796025715956
gzsl: unseen=0.1214 , seen=0.0105 , h=0.0194
____________________________________________________________________________________________________
episode: 77800 loss tensor(0.8270, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.129282368984999
gzsl: unseen=0.1286 , seen=0.0146 , h=0.0263
___________________________________

episode: 82800 loss tensor(1.1912, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.12912300798753165
gzsl: unseen=0.1284 , seen=0.0115 , h=0.0212
____________________________________________________________________________________________________
episode: 83000 loss tensor(1.1233, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.12359022014416521
gzsl: unseen=0.1236 , seen=0.0085 , h=0.0159
____________________________________________________________________________________________________
episode: 83200 loss tensor(0.8464, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.11050340931229301
gzsl: unseen=0.1102 , seen=0.0165 , h=0.0287
____________________________________________________________________________________________________
episode: 83400 loss tensor(1.0542, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class

zsl: 0.09339832456653027
gzsl: unseen=0.0931 , seen=0.0136 , h=0.0237
____________________________________________________________________________________________________
episode: 88400 loss tensor(1.0045, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.09303978180401325
gzsl: unseen=0.0924 , seen=0.0071 , h=0.0132
____________________________________________________________________________________________________
episode: 88600 loss tensor(1.0958, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.1429567114747711
gzsl: unseen=0.1419 , seen=0.0155 , h=0.0279
____________________________________________________________________________________________________
episode: 88800 loss tensor(1.1346, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.10614634716540035
gzsl: unseen=0.1058 , seen=0.0106 , h=0.0192
__________________________________

episode: 93800 loss tensor(1.3658, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.1327472822910579
gzsl: unseen=0.1327 , seen=0.0132 , h=0.0240
____________________________________________________________________________________________________
episode: 94000 loss tensor(1.0225, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.10778231053964543
gzsl: unseen=0.1078 , seen=0.0101 , h=0.0184
____________________________________________________________________________________________________
episode: 94200 loss tensor(1.2668, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.13970934012412678
gzsl: unseen=0.1390 , seen=0.0154 , h=0.0277
____________________________________________________________________________________________________
episode: 94400 loss tensor(0.9703, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class 

zsl: 0.09295012663159945
gzsl: unseen=0.0923 , seen=0.0135 , h=0.0236
____________________________________________________________________________________________________
episode: 99400 loss tensor(1.0576, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.09324430157802455
gzsl: unseen=0.0932 , seen=0.0172 , h=0.0290
____________________________________________________________________________________________________
episode: 99600 loss tensor(1.0512, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.08395336060783168
gzsl: unseen=0.0840 , seen=0.0153 , h=0.0259
____________________________________________________________________________________________________
episode: 99800 loss tensor(0.9718, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.11425801675433471
gzsl: unseen=0.1143 , seen=0.0025 , h=0.0049
_________________________________